In [1]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import RidgeClassifier
from lightgbm import LGBMClassifier

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, auc
import numpy as np
from sklearn.model_selection import cross_val_score
import time as time
import joblib

In [6]:
df = pd.read_csv('precovid.csv')
df.dtypes

Unnamed: 0       int64
Location        object
As             float64
cd             float64
Ni             float64
Fe             float64
pb             float64
zn             float64
Unnamed: 8     float64
Unnamed: 9     float64
Unnamed: 10    float64
dtype: object

In [7]:
df.columns[df.isna().any()]

Index(['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], dtype='object')

##### Label Encoding

In [8]:
df

,Unnamed: 0,Location,As,cd,Ni,Fe,pb,zn,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,0,ramganga_river,0.965635,0.036000,0.162000,2.565000,0.001000,1.441000,NaN,NaN,NaN
1,1,katghar_handpump,0.290895,0.054000,0.105000,0.669000,0.002000,0.657000,NaN,NaN,NaN
2,2,kunwar_saheb_lane,0.873131,0.082000,0.226000,1.623000,0.064000,2.900000,NaN,NaN,NaN
3,3,ramganga,0.411556,0.116000,0.121000,2.704000,0.626000,0.504000,NaN,NaN,NaN
4,4,near_ramganga_handpump,0.327900,0.142000,0.093100,1.701000,0.802000,1.650000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
157,157,warsi_nagar21,0.169062,0.195801,0.266266,0.963756,0.936184,1.144230,NaN,NaN,NaN
158,158,warsi_nagar22,0.521417,0.173434,0.270998,1.071279,0.870394,0.982046,NaN,NaN,NaN
159,159,warsi_nagar23,0.286114,0.188540,0.264798,1.009704,0.896333,1.106447,NaN,NaN,NaN
160,160,warsi_nagar24,0.563254,0.168594,0.237354,0.988376,0.905069,1.134055,NaN,NaN,NaN


In [9]:
len(df['Location'].unique())

162

In [10]:
df.shape

(162, 11)

In [11]:
df.drop(['Unnamed: 0','Location','Unnamed: 8','Unnamed: 9','Unnamed: 10'], axis=True, inplace=True)
df

,As,cd,Ni,Fe,pb,zn
0,0.965635,0.036000,0.162000,2.565000,0.001000,1.441000
1,0.290895,0.054000,0.105000,0.669000,0.002000,0.657000
2,0.873131,0.082000,0.226000,1.623000,0.064000,2.900000
3,0.411556,0.116000,0.121000,2.704000,0.626000,0.504000
4,0.327900,0.142000,0.093100,1.701000,0.802000,1.650000
...,...,...,...,...,...,...
157,0.169062,0.195801,0.266266,0.963756,0.936184,1.144230
158,0.521417,0.173434,0.270998,1.071279,0.870394,0.982046
159,0.286114,0.188540,0.264798,1.009704,0.896333,1.106447
160,0.563254,0.168594,0.237354,0.988376,0.905069,1.134055


In [112]:
# df.drop(['S. No.','River','As','Toxicity'], axis=True, inplace=True)
# df

In [12]:
features = df
# labels = df['Toxicity']

In [13]:
features

,As,cd,Ni,Fe,pb,zn
0,0.965635,0.036000,0.162000,2.565000,0.001000,1.441000
1,0.290895,0.054000,0.105000,0.669000,0.002000,0.657000
2,0.873131,0.082000,0.226000,1.623000,0.064000,2.900000
3,0.411556,0.116000,0.121000,2.704000,0.626000,0.504000
4,0.327900,0.142000,0.093100,1.701000,0.802000,1.650000
...,...,...,...,...,...,...
157,0.169062,0.195801,0.266266,0.963756,0.936184,1.144230
158,0.521417,0.173434,0.270998,1.071279,0.870394,0.982046
159,0.286114,0.188540,0.264798,1.009704,0.896333,1.106447
160,0.563254,0.168594,0.237354,0.988376,0.905069,1.134055


In [14]:
features.describe()

,As,cd,Ni,Fe,pb,zn
count,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000
mean,0.476504,0.102290,0.159426,1.718028,0.415287,1.382367
std,0.306788,0.051267,0.062308,0.754080,0.396782,0.839959
min,0.000420,0.032475,0.084276,0.605907,0.001000,0.454333
25%,0.211020,0.054262,0.106921,0.989608,0.021100,0.678431
50%,0.461745,0.097265,0.140247,1.726899,0.317730,1.224265
75%,0.749188,0.140426,0.222633,2.537107,0.814926,1.643824
max,0.991482,0.198082,0.281405,2.944000,1.055700,3.299533


#### 1. Using LightGBM

In [33]:
start_time=time.time()
lgb_model = joblib.load('lgb_river_model')

y_pred = lgb_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
14.48965072631836 ms


###### 2. Using XGBOOST Classifier

In [34]:
start_time=time.time()
xgb_model = joblib.load('xgb_river_model')

y_pred = xgb_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
15.50912857055664 ms


###### 3. Using RandomForest Classifier

In [35]:
start_time=time.time()
rf_model = joblib.load('rf_river_model')

y_pred = rf_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
35.60137748718262 ms


#### 4. Using Decision Tree

In [36]:
start_time=time.time()
dt_model = joblib.load('dt_river_model')

y_pred = dt_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
 1 0 1 0 1 0 0 1 0 0 0 0 0 0]
7.516622543334961 ms


#### 5. Multinomial Naive Bayes

In [37]:
start_time=time.time()
mnb_model = joblib.load('mnb_river_model')

y_pred = mnb_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
6.99925422668457 ms


#### 6. Gaussian Naive Bayes

In [38]:
start_time=time.time()
gnb_model = joblib.load('gnb_river_model')

y_pred = gnb_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
7.058620452880859 ms


#### 7. SVM

In [40]:
start_time=time.time()
svm_model = joblib.load('svm_river_model')

y_pred = svm_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
5.985736846923828 ms


#### 8. Ridge Classifier

In [41]:
start_time=time.time()
rg_model = joblib.load('rg_river_model')

y_pred = rg_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
6.302833557128906 ms


#### 9. Logistic Regression

In [42]:
start_time=time.time()
logr_model = joblib.load('logr_river_model')

y_pred = logr_model.predict(features)
print(y_pred)

finish_time=time.time()
time_taken=finish_time-start_time
print(time_taken*1000,"ms")

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
5.9967041015625 ms
